[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/grounded-segment-anything-colab/blob/main/grounded-segment-anything-colab.ipynb)

In [ ]:
%cd /content
!pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.18 triton==2.0.0 -U

!apt-get -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/ovseg/resolve/main/sam_vit_h_4b8939.pth -d /content -o sam_vit_h_4b8939.pth

!git clone -b dev https://github.com/camenduru/Grounded-Segment-Anything
%cd /content/Grounded-Segment-Anything
!pip install -r requirements.txt
%cd /content/Grounded-Segment-Anything/GroundingDINO
!pip install .
%cd /content/Grounded-Segment-Anything/segment_anything
!pip install .

In [ ]:
%cd /content/Grounded-Segment-Anything

!npm install -g localtunnel
import subprocess
import threading
import time
import socket
def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('0.0.0.0', port))
        if result == 0:
            break
        sock.close()
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line.decode(), end='')
threading.Thread(target=iframe_thread, daemon=True, args=(7589,)).start()

!python gradio_app.py --share --debug